In [1]:

import glob
import json
import pickle
import re
from distutils.errors import UnknownFileError
from pprint import PrettyPrinter

import pandas as pd
import requests
from apikeys import steamCreds  # * local dependancy
from bs4 import BeautifulSoup
from steam import Steam
from tqdm import tqdm

pd.set_option('display.max_rows', 50)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_columns', 48)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')
pd.set_option('compute.use_numba', True)
pd.set_option('display.date_yearfirst', True)
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.width', None)
pd.set_option('plotting.backend', 'plotly')
pd.set_option("display.show_dimensions", True)
pd.set_option("styler.latex.hrules", True)
tqdm.pandas()
pp = PrettyPrinter().pprint
s = Steam(steamCreds)

In [2]:
# constructing database from scratch

df_vids = pd.concat([pd.read_pickle(i[2:]) for i in glob.glob(
    './pickles/individual/*full.pkl')], ignore_index=True)  # * local
df_vids.dropna(inplace=True)
# minimum number of videos before a game is considered
df_vids = df_vids.groupby("game").filter(lambda x: len(x) >= 10)


In [3]:
df_games = pd.DataFrame(df_vids.game.value_counts()).reset_index()
del df_vids
df_games.drop_duplicates(subset=['index'], inplace=True)
df_games = df_games[~df_games.index.isin(['1996', '2000', '2004', '2009', '2010', '2011', '2012', '2013',
                                        '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'VRChat', 'Youtubers Life'])]
df_games.rename(columns={'index': 'name', 'game': 'count'}, inplace=True)
df_games.set_index('name', inplace=True)
df_games.sample(2)


,count
name,
Dark Souls II,121
Phoenix Point,10


In [4]:
def returnData(data: dict, steam: Steam) -> dict:
    """
    Extracts a dictionary of relevant data from a game's json object

    Args:
        data (dict): dictionary of steam <data> json object for a single game
        steam (Steam): steam api instance initialized with key

    Returns:
        dict: dictionary of relevant information for the game
    """
    container = {
        'appid': ['steam_appid'],
        'requiredAge': ['required_age'],
        'metacritic': ['metacritic', 'score'],
        'price': ['price_overview', 'initial'],
        'windows': ['platforms', 'windows'],
        'mac': ['platforms', 'mac'],
        'linux': ['platforms', 'linux'],
        'releaseDate': ['release_date', 'date'],
        'acheiveCount': ['achievements', 'total'],
        'recs': ['recommendations', 'total'],
        'publishers': ['publishers'],
        'developers': ['developers'],
        'desc': ['about_the_game'],
        'header_image': ['header_image'],
        'genres': None,
        'categories': None
    }
    for key in container:
        if key in ['genres', 'categories']:  # multiple responses
            try:
                _cont = []
                for i in data[key]:
                    _cont.append(i['description'])
                container[key] = _cont[:5]
                continue
            except:  # ! missing data, silently continuing
                container[key] = None
                continue
        try:
            _ = data.copy()
            for i in container[key]:
                _ = _[i]
            container[key] = _
        except KeyError:  # ! missing data, silently continuing
            container[key] = None
            continue
        except AttributeError:  # ! missing data, silently continuing
            container[key] = None
            continue
    try:
        container['publishers'] = container['publishers'][:3]
        container['developers'] = container['developers'][:3]
    except:  # TODO if either is missing; find what error it throws
        pass
    return container


def gameSearch(gamename: str, steam: Steam) -> dict:
    """
    Searches for a game's app_id, returns it's 'data' json object

    Args:
        gamename (str): Title of game
        steam (Steam): steam api instance initialized with key

    Raises:
        NameError: If game cannot be found

    Returns:
        dict: json object of the game's 'data'
    """
    # getting the appid
    #* Attempt 1: Steamspy
    try:
        steamspyids = pd.read_pickle('./pickles/appid.pkl')
    except FileNotFoundError:
        steamspy = pd.DataFrame.from_dict(requests.get(url="https://steamspy.com/api.php", params={"request": "all"}).json(), orient='index')
        steamspyids = steamspy[['appid', 'name']].sort_values('appid').reset_index(drop=True)
        steamspyids.to_pickle('./pickles/appid.pkl')
    
    _ = steamspyids[steamspyids.name.isin([gamename])].appid.to_list()
    if _:
        possibleids = _
    else:
    #* Attempt 2: python.steam.api
        try:
            possibleids = [app['id'] for app in steam.apps.search_games(gamename)['apps']]
    #* Attempt 3: steam search suggestions
        except:
            searchapi = "https://store.steampowered.com/search/suggest"
            with requests.Session() as session:
                params = {"l": "english", "term": gamename, "category1": "998", "cc": "US"} #! something is wrong, occasionally returns nonenglish pages
                response = session.get(searchapi, params=params)
                response.raise_for_status()
                result = BeautifulSoup(response.text, "html.parser").find(
                    'a')
                if result:
                    bundle_id = result.get("data-ds-bundleid")
                    app_id = result.get("data-ds-appid")

                    if bundle_id:
                        name = result.find("div", class_="match_name").get_text()
                        bundle_data = json.loads(
                            re.sub(
                                r"&quot;", '"', result["data-ds-bundle-data"]
                                )
                            )
                        possibleids = [app_id for item in bundle_data["m_rgItems"] for app_id in item["m_rgIncludedAppIDs"]]
                    elif app_id:
                        possibleids = [app_id]
                    else:
                        raise NameError
                else:
                    raise NameError
    if not possibleids:
        raise NameError #! Game not found
    
    finaldata = {}
    for i in range(len(possibleids)):
        idguess = possibleids[i]
        try:
            data = json.loads(steam.apps.get_app_details(idguess))[
                str(idguess)]['data']
            if i == 0: # return first candidate as fallback
                finaldata = data.copy()
            if data['type'] == 'game':
                finaldata = data.copy()
                break
        except:
            continue
    if not finaldata:
        raise NameError #! Data not found
    return finaldata
    


def steamPipeline(df: pd.DataFrame, steam: Steam) -> pd.DataFrame:
    """
    Pipeline to iterate through a dataframe and populate columns. Updates pickled dataframe if available

    Args:
        df (pd.DataFrame): dataframe with game names # ! at column index 0
        steam (Steam): steam api instance initialized with key

    Returns:
        pd.DataFrame: Original dataframe with new columns, missing data silently filled with nan
    """
    df = df.reindex(
        columns=df.columns.tolist() + ['appid', 'requiredAge', 'metacritic', 'price', 'windows', 'mac', 'linux', 'releaseDate', 'acheiveCount', 'recs', 'publishers', 'developers', 'desc', 'header_image', 'genres', 'categories'])
    try:
        df_old = pd.read_pickle('./pickles/df_games.pkl')
        df_old.set_index('name', inplace=True)
        df = df.fillna(df_old)
    except FileNotFoundError:  # first run
        pass
    try:
        nonsteam = pickle.load(open('./pickles/nonsteamlist.pkl', 'rb'))
    except FileNotFoundError:  # first run
        nonsteam = []
    t = tqdm(df.itertuples(), total=df.shape[0])
    errors = 0
    for row in t:
        name = row[0]
        # skipping scraped rows 
        if (df.loc[name, :].isna().sum() < 16): #TODO change to a lower number later (~8?)
            continue
        elif name in nonsteam:
            continue
        try:
            data = gameSearch(name, steam)
        except NameError:  # ! Game cannot be found, continuing
            errors += 1
            t.set_postfix({'Total errors': errors, "Last missing game": name})
            continue
        result = returnData(data, steam)
        for column in result:
            if type(result[column]) is list:
                df[column] = df[column].astype('object')
            df.at[name, column] = result[column]
    nonsteam = 0
    filter = df.isna().sum(axis=1) == 16 # empty games
    nonsteam = df[filter].index.to_list() # filter so don't search again
    df = df[~filter]
    df.reset_index(inplace=True)
    pickle.dump(nonsteam, open('./pickles/nonsteamlist.pkl', 'wb'))
    df.to_pickle('./pickles/df_games.pkl')
    return df


In [5]:
df_games = steamPipeline(df_games, s)


100%|██████████| 1050/1050 [00:43<00:00, 24.08it/s, Total errors=28, Last missing game=Desperados III]              


In [6]:
df_games['releaseDate'] = pd.to_datetime(
    df_games.releaseDate, errors='coerce')


In [7]:
df_games.sample(5)


,name,count,appid,requiredAge,metacritic,price,windows,mac,linux,releaseDate,acheiveCount,recs,publishers,developers,desc,header_image,genres,categories
284,DARK SOULS: REMASTERED,34,570940,17,84,3999,True,False,False,2018-05-23,41,47768,"[FromSoftware, Inc, Bandai Namco Entertainment]",[QLOC],"Then, there was fire. Re-experience the critically acclaimed, genre-defining game that started it all. Beautifully remastered, return to Lordran i...",https://cdn.akamai.steamstatic.com/steam/apps/570940/header.jpg?t=1668145065,[Action],"[Single-player, Multi-player, Steam Achievements, Full controller support, Remote Play on Phone]"
210,Tropico 4,55,57690.00,0,78.00,84900.00,True,False,False,NaT,70.00,7587.00,[Kalypso Media Digital],[Haemimont Games],"Мир меняется, и Tropico идет в ногу со временем — географические страны вздымаются и падают, мировой рынок во власти новых игроков с новыми требов...",https://cdn.akamai.steamstatic.com/steam/apps/57690/header.jpg?t=1665482549,"[Симуляторы, Стратегии]","[Для одного игрока, Достижения Steam, Коллекционные карточки]"
218,Wolfenstein: The New Order,54,201810.00,16,81.00,1999.00,True,False,False,2014-05-19,50.00,34921.00,[Bethesda Softworks],[MachineGames],"Wolfenstein®: The New Order reignites the series that created the first-person shooter genre. Developed by MachineGames, a studio comprised of a s...",https://cdn.akamai.steamstatic.com/steam/apps/201810/header.jpg?t=1630605243,[Action],"[Single-player, Steam Achievements, Partial Controller Support, Steam Leaderboards]"
101,2018,105,645630.00,0,NaN,1999.00,True,True,False,2017-07-28,60.00,40718.00,[PlayWay S.A.],[Red Dot Games],"<a href=""https://store.steampowered.com/app/1573280/WW2_Rebuilder/url]"" target=""_blank"" rel="""" id=""dynamiclink_3"" >https://store.steampowered.com...",https://cdn.akamai.steamstatic.com/steam/apps/645630/header.jpg?t=1674471914,"[Racing, Simulation]","[Single-player, Steam Achievements, Steam Trading Cards, Steam Workshop, Partial Controller Support]"
625,Left 4 Dead,11,500.00,0,89.00,999.00,True,True,False,2008-11-17,73.00,37564.00,[Valve],[Valve],"From Valve (the creators of Counter-Strike, Half-Life and more) comes Left 4 Dead, a co-op action horror game for the PC and Xbox 360 that casts u...",https://cdn.akamai.steamstatic.com/steam/apps/500/header.jpg?t=1673386755,[Action],"[Single-player, Multi-player, Co-op, Steam Achievements, Full controller support]"


In [8]:
df_games.isna().sum(axis=0)


name              0
count             0
appid             0
requiredAge       0
metacritic      280
price            65
windows           0
mac               0
linux             0
releaseDate      22
acheiveCount    149
recs             34
publishers        0
developers        0
desc              0
header_image      0
genres            1
categories        0
Length: 18, dtype: int64

In [9]:
df_games[['name', 'desc']].sample(50)

,name,desc
332,Control,"<img src=""https://cdn.akamai.steamstatic.com/steam/apps/870780/extras/Control_UE_Steam_Vanity.jpg?t=1655979558"" /><br><br><strong>Control Ultimate..."
518,Total War: Attila,"Against a darkening background of famine, disease and war, a new power is rising in the great steppes of the East. With a million horsemen at his ..."
370,Remember Me,"Neo-Paris. 2084.<br><br>Personal memories can now be digitized, bought, sold and traded. The last remnants of privacy and intimacy have been swept..."
326,Anthem,"<img src=""https://cdn.akamai.steamstatic.com/steam/apps/787850/extras/01_STORY.png?t=1666950860"" /><br><img src=""https://cdn.akamai.steamstatic.co..."
411,Staxel,"Staxel is a creative farming and village life game with a cast of quirky characters. Build your farmhouse, help villagers, take to the wilds to fi..."
344,Transistor,"From the creators of <strong>Bastion</strong>, <strong>Transistor</strong> is a sci-fi themed action RPG that invites you to wield an extraordinar..."
314,RuneScape,"<img src=""https://cdn.akamai.steamstatic.com/steam/apps/1343400/extras/RS3-01m_Section_header_image_1-WELCOME_TO_RUNESCAPE-EN.png?t=1668764247"" />..."
418,Turmoil: The Heat is On,<i>Turmoil - The Heat Is On</i> is a brand new campaign with tons of oil to dig up and convert into cash! It is designed to make you feel right at...
187,Injustice 2,Power up and build the ultimate version of your favourite DC legends in INJUSTICE 2. <br />\r\n•EVERY BATTLE DEFINES YOU: With every match you'll ...
343,StarDrive 2,"StarDrive 2 is an exciting and evolutionary step forward for the StarDrive franchise. In this eagerly anticipated sequel, the core mechanics have..."
